In [1]:
# show more than one output in cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# plot charts in our notebook
%matplotlib inline

In [2]:
from torchvision import models as tvm
import pretrainedmodels as ptm
models = (
    ptm.alexnet, # gets maximum recursion limit exceeded exceptions
    ptm.se_resnet50,
    ptm.se_resnet101,
    ptm.inceptionresnetv2,
    ptm.inceptionv4,
    ptm.vgg16,
    ptm.vgg19,
    tvm.resnet101,
    ptm.senet154,
    ptm.nasnetalarge, # calculated output size is too small
)

domain = [{'name': 'batch_size', 'type': 'discrete', 'domain': (16, 24, 32, 48, 64)},]

adam_domain = domain + [
    {'name': 'adam_lr', 'type': 'continuous', 'domain': (0.001, 0.1)},
    {'name': 'adam_beta1', 'type': 'continuous', 'domain': (0.8, .99)},
    {'name': 'adam_beta2', 'type': 'continuous', 'domain': (0.95, .9999)},
    {'name': 'adam_wtdecay', 'type': 'continuous', 'domain': (0, 1)}
]
#default_input = [32, 0.001, 0.9, 0.999, 0] # TODO: have to figure out how to do this

SGD_domain = domain + [
    {'name': 'lr', 'type': 'continuous', 'domain': (0.001, 0.1)},
    {'name': 'momentum', 'type': 'continuous', 'domain': (0.5, .99)},
    {'name': 'weight_decay', 'type': 'continuous', 'domain': (0, 1)}
]

In [3]:
import src.utils as utils
from src.trainable import Trainable
import torch


def build_model(model_fn):
    print(model_fn)
    if 'pretrainedmodels' in model_fn.__module__:
        model = model_fn(num_classes=1000, pretrained='imagenet')
    else:
        model = model_fn(pretrained=True)
    return model
    

def f(x):
    """ Value function to minimize for bayesian optimization """
    val_acc = train(
        batch_size=int(x[:,0]),
        adam_lr=float(x[:,1]),
        adam_b1=float(x[:,2]),
        adam_b2=float(x[:,3]),
        adam_wtdecay=float(x[:,4]),
    )
    
    return val_acc


CRITERION = torch.nn.CrossEntropyLoss()


def make_key_from_params(params):
    """ Makes a unique key (as a tuple) from a given list of parameters.
    """
    return tuple(round(param, 10) for param in params)


def train(batch_size, adam_lr, adam_b1, adam_b2, adam_wtdecay):
    input_params = locals()  # the parameters will become the key
    
    
    image_size = utils.determine_image_size(utils.get_model_name_from_fn(chosen_model))
    dataloaders = utils.get_train_val_dataloaders(
        datadir=data_dir,
        val_proportion=0.15,
        image_size=image_size, 
        batch_size=batch_size
    )
    
    model = build_model(chosen_model)
    utils.fit_model_last_to_dataset(model, dataloaders['train'].dataset)
    
    optimizer = torch.optim.Adam(model.parameters(), 
                                 adam_lr, (adam_b1, adam_b2), adam_wtdecay)
    
    trainable = Trainable(dataloaders, model, CRITERION, optimizer, outdir=outdir)
    acc = trainable.train(50, early_stop_limit=6, verbose=False)
    
    # store this train iteration in a dictionary for later
    global trainables
    key = make_key_from_params(input_params)
    trainables.append({key: trainable})
    return acc
    



## Do BO on all models on both datasets. Git push at each stage

In [4]:
import os
def make_outdir_name(datadir, model_name, prepend="", append=""):
    dataset_name = os.path.basename(datadir)
    return os.path.join(prepend, dataset_name, model_name, append) 

In [5]:
from GPyOpt.methods import BayesianOptimization
from predict import create_predictions
import subprocess
import sys
sys.setrecursionlimit(1500)

EXPERIMENTS_DIR = os.path.join("experiments", "bayes_opt_2")

for model_fn in models:
    global chosen_model
    chosen_model = model_fn
    
    for data_index, datadir in enumerate(('data/die_vs_all_tt', 'data/4_class_tt')):
        global iteration  # keep track of our optimization iterations for directory output
        iteration = 0   # but reset to 0 each train run
        global data_dir
        data_dir = datadir
        # make an output directory using the model, dataset, and BO iteration
        global outdir
        outdir = make_outdir_name(datadir, utils.get_model_name_from_fn(chosen_model), 
                                  prepend=EXPERIMENTS_DIR,
                                  append=str(iteration))
        
        # storage of all the trainables produced by BO
        global trainables
        trainables = {}
        
        try:
            # define and optimize the problem
            problem = BayesianOptimization(
                f=f,
                domain=domain,
                maximize=True
            )
            problem.run_optimization(max_iter=10)
            
            # plot the results
            problem.plot_acquisition()
            problem.plot_convergence()
            
            # save the best trainable
            best_params = problem.x_opt
            key = make_key_from_params(best_params)
            best_trainable = trainables[key]
            best_trainable.save()
            
            # evalute on the test set using the best model
            predictions_file = create_predictions(
                outdir=outdir,
                subset='test',
                data_dir=data_dir,
                model=best_trainable.model
            )
            create_all_metrics(predictions_file, outdir, "test")
            
        # if something bad happens, skip it so we can let the others run
        except Exception as e:
            print('Skipping because', e)
#             import traceback
#             traceback.print_exc()
            continue
            
        # commit & push only if we can connect to internet
        subprocess.check_call(['git', 'add', 'experiments'])
        subprocess.check_call(['git', 'commit', '-am', f'Results from {str(chosen_model).split()[1]} {datadir}'])
        subprocess.check_call(['git', 'push'])

<function alexnet at 0x7f573fc30e18>


Epoch 1, Train:  10%|▉         | 96/987 [00:16<02:35,  5.73images/s]

Sigint caught!
Training will stop after this epoch and the best model so far will be saved.
OR press Ctrl-C again to quit immediately without saving.


Stopping...


SystemExit: 1

In [ ]:
_ = subprocess.check_call(["spd-say", "Your code has finished running"])
_ = subprocess.check_call(['git', 'commit', '-am', "BO final commit"])
_ = subprocess.check_call(['git', 'push'])
    